In [1]:
import numpy as np
import os
import shutil
import sys

In [2]:
datasets = '../datasets/'
glove_file_location = os.path.join(datasets, 'glove.6B/glove.6B.100d.txt')

In [3]:
%%time 
# Process the Glove vectors into a dictionary
embeddings_index = {}
max_database_size = 400000
with open(glove_file_location, encoding='utf-8') as f:
    for line in f:
        parsed_line = line.split()
        # The explicit string conversion below seems to fix a weird unicode runtime issue
        word = str(parsed_line[0]) 
        vector = np.array(parsed_line[1:], dtype='float32')
        embeddings_index[word] = vector
        if len(embeddings_index) >= max_database_size:
            break

print('Loaded {0} words from database.'.format(len(embeddings_index)))
    

Loaded 400000 words from database.
CPU times: user 13.6 s, sys: 424 ms, total: 14.1 s
Wall time: 14.2 s


In [4]:
word  = 'their'
if word in embeddings_index:
    print(embeddings_index[word])


[ 0.17137   -0.33437    0.11471   -0.52008   -0.79816    0.48592
 -0.73885    0.28653   -0.6273     0.26786   -0.32081    0.11757
  0.61241    0.25964   -0.21165   -0.63307    0.40135    0.26175
 -0.16827   -0.0092904  0.35687   -0.31246    0.79867    0.54921
  0.17423   -0.30704   -0.23997   -0.64167    0.36679   -0.27327
  0.64719    0.32952   -0.46198    0.057414  -0.088337  -0.027794
 -0.074753   0.52992   -0.48819   -0.17822    0.045592  -0.39832
  0.5287     0.01556    0.23284   -0.011029  -0.15533   -0.065942
  0.22879   -1.0154    -0.2172     0.26772   -0.32618    1.6999
  0.055699  -2.586      0.75177   -0.76514    1.9965     0.28489
 -0.1825     1.3012    -0.074158   0.22211    0.8391     0.12388
  0.4363     0.4045     0.25744   -0.63188   -0.34572   -0.097293
  0.14742   -1.5755    -0.098063   0.36815   -0.83086   -0.41775
 -1.3509     0.11379    0.76722    0.75707   -0.24826    0.4637
 -1.8673    -0.46703   -0.01854    0.010588  -0.76759    0.21896
 -0.46664    0.32206    

#### Tokenize raw imdb database

- Use 10000 words to tokenize
- Use 200 training samples only
- Use 10000 validation samples
- Use 100 words from each sample 